In [115]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI,Ollama
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [116]:
## Pdf reader this will load the data 
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('./resourcev1.pdf')
docs=loader.load()

In [117]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [81]:
## Vector Embedding And Vector Store
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents[:],OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://host.docker.internal:11434" 
))

In [ ]:


prompt=ChatPromptTemplate.from_template(
    """
question: {query}

    ## Core Role

You are an AI diagnostic assistant that extracts clinically relevant information from patient symptom descriptions using the pdf (Merck Manual), with web search fallback capability.

## Primary Diagnostic Protocol

### Input Processing

**Extract from patient text:**

- Symptom(s), duration, severity (1-10), modifying factors
- Infer vital sign implications (e.g., "can't catch breath" → tachypnea)
- Flag emotional urgency cues ("worst pain ever", "never felt this before")

### Standard Response Format

**1. Structured Symptom Summary**

```
Interpreted as: [Medical terminology translation]
• Chief complaint: [Primary symptom]
• Duration: [Timeframe]
• Severity: [1-10 scale if provided]
• Associated symptoms: [List]
• Risk factors: [If mentioned]

```

**2. Likelihood Matrix**

```
│ Common (PCP-manageable)    │ Urgent (ER-needed)        │
├───────────────────────────┼──────────────────────────┤
│ 1. [Most likely diagnosis] │ A. [Emergency condition]  │
│ 2. [Second likely]         │ B. [Second emergency]     │
│ 3. [Third consideration]   │                          │

```

**3. Next-Step Decision Tree**

```
« If [mild features] → "Try [conservative measure] for [timeframe]"
« If [red flags] → "Seek care at [Urgent Care/ER] today"
« Either way → "Recommend checking: [specific tests]"

```

**4. Merck Manual Citations**

- Reference: [Chapter/Section/Page numbers used]

### Emergency Response Protocol

🚨 **Immediate triggers**: Chest pain + cardiac risk, severe SOB, neurological symptoms, severe headache, active bleeding, suicidal ideation

**Emergency template:**
⚠️ URGENT: These symptoms require immediate medical evaluation
→ Consider: Emergency services or immediate physician contact
→ Concerning features: [List specific red flags]

## Web Search Protocol

*Activate when Merck Manual lacks sufficient coverage*

### Search Activation Triggers

- Symptoms not adequately covered in Merck Manual
- Rare conditions suspected
- Recent medical developments needed
- Specific diagnostic criteria required

### Safety Check Before Web Search

**Assess if the gap is due to:**

- (a) Patient's colloquial description needing reinterpretation?
- (b) Very rare condition requiring specialist input?
- (c) Newly recognized/evolving medical knowledge?

**Actions:**

- If (a): Re-analyze using medical synonyms: [suggest 3 variations]
- If (b): "Recommend [specialty] consultation for [rare condition class]"
- If (c): Proceed to web search with medical constraints

### Web Search Parameters

**Authorized Sources Only:**

- Tier 1: .gov, .edu, WHO, CDC, NIH, FDA
- Tier 2: Academic medical centers, professional societies
- Tier 3: Peer-reviewed journals, Cochrane reviews

**Search Strategy:**

- Keywords: [Auto-generated from symptoms + "diagnosis criteria" + "differential diagnosis"]
- Time filter: Last 5 years (unless historical context needed)
- Exclude: Patient forums, commercial sites, non-medical sources

### Web Search Output Format

```
📊 **Web-Retrieved Clinical Guidance** (Verify before clinical use)

**Condition**: [Medical condition name]
**Diagnostic Criteria**:
• [Criterion 1]
• [Criterion 2]
• [Criterion 3]

**Differential Considerations**: [List alternatives found]

**Source Confidence**:
□ Consensus (multiple agreeing authoritative sources)
□ Emerging (single authoritative source)
□ Conflicting data (note discrepancies)

**Primary Source**: [Organization] | [URL] | [Date]
**Additional Sources**: [List if multiple used]

**Merck Manual Gap Explanation**: [Why this wasn't in offline manual - e.g., recent emergence, rare condition, updated criteria]

**Integration with Standard Format**: [Apply same likelihood matrix and patient explanation structure using web-sourced information]

```

### Web Search Quality Controls

- Maximum 3 web sources per query
- Always include publication dates
- Flag any conflicting information between sources
- Maintain same safety protocols as primary diagnostic process
- Include disclaimer: "Web-sourced information requires clinical verification"

    """
)


In [111]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0xffff680f2ef0>, search_kwargs={})

In [84]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.llms import Ollama


llm=Ollama(model="Gemma3:1b",base_url="http://host.docker.internal:11434",temperature=0.9)

documents_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)

#qa_chain=RetrievalQA.from_chain_type(llm=llm,retriever=db.as_retriever(),chain_type_kwargs={"prompt":prompt})




In [85]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,documents_chain)

In [112]:
retrieval_chain({"query":"i have chest pain"})

/tmp/ipykernel_14614/3932619813.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieval_chain({"query":"i have chest pain"})


{'query': 'i have chest pain',
 'result': 'Chest pain can be exacerbated by respiration, movement, or palpation of the chest. About 15% of patients with acute MI have chest tenderness on palpation. Nitroglycerin may relieve pain of both myocardial ischemia and noncardiac smooth muscle spasm.'}

## Part 2


In [ ]:
%pip install -qU langchain-tavily

Note: you may need to restart the kernel to use updated packages.


In [90]:
import os
os.environ["TAVILY_API_KEY"]="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"

In [91]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [92]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [93]:
tool.invoke({"query": "i have chest pain"})

{'query': 'i have chest pain',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Chest pain: 27 causes, symptoms, and when to see a doctor',
   'url': 'https://www.medicalnewstoday.com/articles/321650',
   'content': 'Chest pain can stem from a heart problem, but other possible causes include a lung infection, muscle strain, a rib injury, or a panic attack. Some of these are serious conditions and need medical',
   'score': 0.67063034,
   'raw_content': None},
  {'title': 'Chest Pain: What It Feels Like, Causes & Treatment - Cleveland Clinic',
   'url': 'https://my.clevelandclinic.org/health/symptoms/21209-chest-pain',
   'content': 'Giving Careers Search ClevelandClinic.org Find A Doctor Locations & Directions Patients & Visitors Health Library Institutes & Departments Appointments Home/ Health Library/ Symptoms/ Chest Pain Advertisement Advertisement Chest Pain Chest pain has many causes, like issues in your heart, lungs or digestive system. Some c

In [94]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "euro 2024 host nation"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

{"query": "euro 2024 host nation", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "UEFA Euro 2024 - Wikipedia", "url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024", "content": "It was the third time that European Championship matches were played on German territory, and the second time in reunified Germany, as West Germany hosted the 1988 tournament, and four ma


## Part 3

In [113]:
# === Step 3: Wrap Tavily in a Tool ===
from langchain.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.chains import RetrievalQA
from langchain.tools.retriever import create_retriever_tool
from langchain_tavily import TavilySearch
from langchain.tools import Tool
tavily_tool = Tool(
    name="tavily_search",
    description="Search the web for general information using Tavily.",
    func=TavilySearch(max_results=1, topic="general")
)

from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

# === Step 4: Create Tool List ===
tools = [retriever_tool, tavily_tool]

# from langchain.agents import create_openai_tools_agent
# agent=create_openai_tools_agent(llm,tools,prompt)

# === Step 5: Initialize Agent ===
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    prompt=prompt
    
)

# === Step 6: RetrievalQA Chain (used before fallback) ===
retrieval_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# === Step 7: Hybrid Logic ===
def hybrid_query(input_text: str):
    response = retrieval_chain.invoke({"query": input_text})
    answer = response.get("result", "").strip()

    if not answer or "I don't know" in answer or "No relevant" in answer:
        print("Fallback to Tavily via agent...")
        return agent.run(input_text)
    else:
        print("Answer from Retrieval QA:")
        return answer

# === Example Usage ===
result = hybrid_query("I have very sharp pain in left side of my chest, and shortness of breath")
print(result)

Answer from Retrieval QA:
I understand you’re asking about the possible causes of sharp chest pain and shortness of breath, given the context provided.

Based on the information, several possibilities could be considered:

*   **Cardiac Issues:** The context mentions pericarditis and the possibility of similar chest pain to myocardial inflammation or ischemia. It also points out that pericarditis can be triggered by tamponade.
*   **Pulmonary Issues:** The mention of dyspnea (shortness of breath) suggests a potential issue with the lungs.
*   **Gastrointestinal Issues:** The context indicates that painful stimuli from thoracic organs can be referred to the upper extremities, which could be related to digestive discomfort.
*   **Neurological Issues:** The description of “discomfort” and the potential for stabbing pain could be related to neurological problems.

**Disclaimer:** *I am an AI Chatbot and not a medical professional. This information is for general knowledge and informational

NameError: name 'retriever_tool' is not defined

NameError: name 'retriever_tool' is not defined

NameError: name 'retriever_tool' is not defined

NameError: name 'retriever_tool' is not defined

NameError: name 'retriever_tool' is not defined

NameError: name 'retriever_tool' is not defined